<a href="https://colab.research.google.com/github/rajdeepbanerjee-git/Data_Augmentation_LLM/blob/main/MA2_p2_Phi3_run_spam_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

We will now load the saved FN cases created using another notebook. These spam cases will used for few shot learning to generate more such samples that can be used for data augmentation.

In [ ]:
FN_cases_df = pd.read_csv("/content/FN_cases.csv")
FN_cases_df.head()

,sms,label
0,22 days to kick off! For Euro2004 U will be ke...,1
1,"Twinks, bears, scallies, skins and jocks are c...",1
2,Will u meet ur dream partner soon? Is ur caree...,1
3,Hello darling how are you today? I would love ...,1
4,Check Out Choose Your Babe Videos @ sms.shsex....,1


In [ ]:
import pprint as pp
import random

def generate_prompt(FN_cases_df, n_examples, n_out):

    prompt_header = f"""
    Consider yourself as a synthetic data generator.
    I will be giving you 'spam' messages from a ham-spam dataset.
    These specific spam messages are where the model did not perform well.
    You need to generate more such spam messages for the model to retrain.
    Here are {n_examples} examples of the spam messages:
    """
    prompt_footer = f"""
    Now, generate {n_out} more such spam messages.
    Output each generated meassage in following format:
    <START> <genrated_message> <END>
    """

    ind_list = random.choices(FN_cases_df.index, k= n_examples)

    prompt_body = """\n"""
    for i, j in enumerate(ind_list):
      prompt_body = prompt_body + f"Example {i}:\n" + FN_cases_df['sms'].loc[j] + "\n"

    final_prompt = prompt_header + prompt_body + prompt_footer

    return final_prompt

In [ ]:
test_prompt = generate_prompt(FN_cases_df = FN_cases_df, n_examples = 5, n_out = 10)
print(test_prompt)


    Consider yourself as a synthetic data generator. 
    I will be giving you 'spam' messages from a ham-spam dataset. 
    These specific spam messages are where the model did not perform well. 
    You need to generate more such spam messages for the model to retrain. 
    Here are 5 examples of the spam messages:
    
Example 0:
Hi its LUCY Hubby at meetins all day Fri & I will B alone at hotel U fancy cumin over? Pls leave msg 2day 09099726395 Lucy x Calls£1/minMobsmoreLKPOBOX177HP51FL

Example 1:
Dont forget you can place as many FREE Requests with 1stchoice.co.uk as you wish. For more Information call 08707808226.

Example 2:
Hi ya babe x u 4goten bout me?' scammers getting smart..Though this is a regular vodafone no, if you respond you get further prem rate msg/subscription. Other nos used also. Beware!

Example 3:
FREE2DAY sexy St George's Day pic of Jordan!Txt PIC to 89080 dont miss out, then every wk a saucy celeb!4 more pics c PocketBabe.co.uk 0870241182716 £3/wk

Example 

#### LLM Generation

Now, we will generate similar spam messages using LLM. Because it is open-source and supposed work well even with its relatively small number of parameters, we will use phi3-mini. It takes up < 3 GB of VRAM when loaded in 4 bit quantization, hence we can run it in free T4 colab GPUs!

Let us now load the relevant packages needed to run it.

In [ ]:
!pip install langchain huggingface_hub
!pip install -U accelerate peft bitsandbytes
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,BitsAndBytesConfig

In [ ]:
# to load the model in 4bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model_id = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             trust_remote_code=True
                                            )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,temperature=0.5, max_new_tokens=4000)
hf = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """<|user|>\n{question}<|end|>\n<|assistant|>"""
prompt = PromptTemplate.from_template(template)

# creating chain of prompt hf pipeline
chain = prompt | hf


In [ ]:
# now we run the llm using our prompt and save the results

import os
from tqdm import tqdm

file_path = "/content/llm_output.txt"
for c in tqdm(range(5)):
    prompt = generate_prompt(FN_cases_df = FN_cases_df, n_examples = 5, n_out = 10)
    output = chain.invoke({"question": prompt})

    if os.path.exists(file_path):
      with open(file_path, 'a') as file:
        file.write(output + '\n ####')

    else:
      with open(file_path, 'w') as file:
        file.write(output + '\n ####')

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 20%|██        | 1/5 [00:59<03:58, 59.61s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 40%|████      | 2/5 [01:43<02:30, 50.26s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sam